In [1]:
import xgboost as xgb
import sys,random
import pickle
import os
import numpy as np

In [2]:
# import modin.pandas as pd
import pandas as pd

In [3]:
if not os.path.exists('featurescore'):
    os.mkdir('featurescore')
if not os.path.exists('model'):
    os.mkdir('model')
if not os.path.exists('preds'):
    os.mkdir('preds')

使用旧数据集 train集的维度为：  
使用新数据集 train集的维度为：6702

### 加载训练数据(无tag)

In [6]:
%%time
data_date = pd.read_csv('../../preprocess_data/train_x_date.csv').drop(columns=['id','loan_hour'])
data_raw = pd.read_csv('../../preprocess_data_new/train_ax_nodup.csv',nrows=33465).drop(columns=['id','loan_dt','tag'])
data_null = pd.read_csv('../../preprocess_data_new/train_ax_row_null.csv',nrows=33465).drop(columns=['id'])
data_label = pd.read_csv('../../preprocess_data/train_y_33465.csv',usecols=['label'])
data = pd.concat([data_date,data_raw,data_null],axis=1)
x_train = data.fillna(-1).values
y_train = data_label.values.ravel()

CPU times: user 55.8 s, sys: 3.56 s, total: 59.3 s
Wall time: 1min 1s


In [7]:
x_train.shape

(33465, 6701)

### 加载unlabel数据(无tag）

In [4]:
%%time
unlabel_date = pd.read_csv('../../preprocess_data/unlabel_x_date.csv').drop(columns=['id','loan_hour'])
unlabel_raw = pd.read_csv('../../preprocess_data_new/train_ax_nodup.csv',skiprows=range(1,33466)).drop(columns=['id','loan_dt','tag'])
unlabel_null = pd.read_csv('../../preprocess_data_new/train_ax_row_null.csv',skiprows=range(1,33466)).drop(columns=['id'])
unlabel = pd.concat([unlabel_date,unlabel_raw,unlabel_null],axis=1)
x_test = unlabel.fillna(-1).values

CPU times: user 1min 55s, sys: 9.84 s, total: 2min 5s
Wall time: 2min 11s


In [5]:
x_test.shape

(66535, 6701)

### 定义学习器

In [8]:
def SelectModel(model_name):
    if model_name == 'GBC':
        from sklearn.ensemble import GradientBoostingClassifier
        model = GradientBoostingClassifier(loss='deviance',
                                           learning_rate =0.1,
                                           n_estimators=300,
                                           subsample=0.9,
                                           max_depth=3,
#                                            verbose=1,
                                          random_state=2018)
    elif model_name == 'XGB':
        from xgboost import XGBClassifier

        model = XGBClassifier(max_depth=6,
                              learning_rate =0.04, 
                              booster='gbtree',
                              objective='binary:logistic',
                              early_stopping_rounds=100,
                              scale_pos_weight=float(len(y_train)-np.sum(y_train))/float(np.sum(y_train)),
                              eval_metric='auc',
                              gamma=1,
                              reg_lambda=1,
                              subsample=0.9,
                              min_child_weight=1,
                              seed=2018,
                              silent=False,
                              n_jobs=24,
                              num_boost_round =300
                             )
    elif model_name == 'RFC':
        from sklearn.ensemble import RandomForestClassifier
        model = RandomForestClassifier(n_estimators=1500,
                                       n_jobs =36,
                                       max_features='sqrt',
                                       class_weight='balanced',
#                                        verbose =1,
                                       random_state=2018)
    elif model_name == 'LGB':
        from lightgbm import LGBMClassifier
        model = LGBMClassifier(boost='gbdt',
                    num_leaves=135, 
                    scale_pos_weight=float(len(y_train)-np.sum(y_train.ravel()))/float(np.sum(y_train.ravel())),
                    max_depth=-1,
                    learning_rate=.05,
                    max_bin=200,
                    min_data_in_leaf= 60,
                    objective='binary',
                    metric='auc',
                    num_threads=32,
                    slient=False,
                    num_boost_round =300)
    else:
        pass
    return model

In [ ]:
%%time
import joblib
model_list = ['XGB','RFC','LGB']
unlabel_pred_list = []

for model in model_list:
    print(model)
    clf = SelectModel(model)
    clf.fit(x_train,y_train)
    joblib.dump(clf,'./model/{}'.format(model))  
    

XGB
RFC
LGB


/usr/local/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [6]:
import joblib
import os
model_list = os.listdir('./model')
clf = joblib.load('./model/{}'.format(model_list[0]))
unlabel_pred = clf.predict_proba(x_test)[:,1]

for model in model_list[1:]:
    clf = joblib.load('./model/{}'.format(model))
    unlabel_pred += clf.predict_proba(x_test)[:,1]
unlabel_pred = unlabel_pred/len(model_list)
# 输出
unlabel_id = pd.read_csv('../../preprocess_data/unlabel_x_date.csv',usecols=['id']).values.ravel()
pred = pd.DataFrame({'id':unlabel_id,'prob':unlabel_pred})
pred.to_csv('./preds/unlabel_pred.csv',index=False,encoding='utf-8')